Dr. Shoemaker will continue to allow corrections for misunderstandings

But not extensions

Today's lecture:

Interaction effects, mixed-effects modeling, and controlling for random effects: all the same thing phrased differently

My homework found solutions that are pretty far away from a, b

Why are my ODEs getting solved weirdly?

Did the rest of the homework correctly I think

Review of what we've covered so far:

  basic coding & plotting
  regression - linear & nonlinear (gradient descent)

  Statistics - t test, u null distributions, and general strategies for handling untraditional data

  Bootstrapping p values

Today: wrap up data analytics with mixed effects modeling

Next week: transition to general ODE dynamics

Linear models:

  Most basic is simply y = mx + b (continuous or discrete measurements)

  We can expand linear models to include more than one factor

  y = m1x1 + m2x2 + b
  
  b is still a column of 1s

  still completely linear

  This is an or interpretation

Due by end of class: use linear algebra code and cars data (hw2) to analyze how mpg depends on cylinders and the weight of the car

cylinders 6 or greater is one group

weight greater than 3.4 is one group

In [41]:
import pandas as pd
import numpy as np
from numpy.linalg import inv

weightbinary = []
cylbinary = []

from google.colab import files

dat = pd.read_csv("cars.csv")

mpg = dat["mpg"].values.tolist()
weight = dat["wt"].values.tolist()
cyl = dat["cyl"].values.tolist()

for k in range(len(mpg)):
  if weight[k] >= 3.4:
    weightbinary.append(1)
  else:
    weightbinary.append(0)
  if cyl[k] >= 6:
    cylbinary.append(1)
  else:
    cylbinary.append(0)

npmpg = np.array(mpg)
npweight = np.array(weightbinary)
npcyl = np.array(cylbinary)

intercept = np.ones(len(mpg))

matrix = np.vstack((intercept, npweight, npcyl)).T

tmp = inv(np.matmul(np.transpose(matrix), matrix))

INTERCEPT, SLOPEw, SLOPEcyl = tmp @ np.transpose(matrix) @ mpg

print('Intercept is:', INTERCEPT, "slope w is:", SLOPEw, 'slope cyl is:', SLOPEcyl)

Intercept is: 26.663636363636368 slope w is: -4.111250000000005 slope cyl is: -6.8836363636363656


The above values are correct for binary linear algebra. Going to do another exercise after this

In [42]:
import pandas as pd
import numpy as np
from numpy.linalg import inv

weightbinary = []
cylbinary = []

from google.colab import files

dat = pd.read_csv("cars.csv")

mpg = dat["mpg"].values.tolist()
weight = dat["wt"].values.tolist()
cyl = dat["cyl"].values.tolist()

npmpg = np.array(mpg)
npweight = np.array(weight)
npcyl = np.array(cyl)

intercept = np.ones(len(mpg))

matrix = np.vstack((intercept, npweight, npcyl)).T

tmp = inv(np.matmul(np.transpose(matrix), matrix))

INTERCEPT, SLOPEw, SLOPEcyl = tmp @ np.transpose(matrix) @ mpg

print('Intercept is:', INTERCEPT, "slope w is:", SLOPEw, 'slope cyl is:', SLOPEcyl)

Intercept is: 39.68626148025295 slope w is: -3.190972138983728 slope cyl is: -1.5077949682598129


The above script solves the system for a function where weight and cylinders' exact values are used to create a linear function. IE not binary.



In [43]:
import statsmodels.api as sm
import math

matrix = np.vstack((intercept, npweight, npcyl)).T
model = sm.OLS(mpg, matrix).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                     70.91
Date:                Tue, 11 Oct 2022   Prob (F-statistic):           6.81e-12
Time:                        20:47:21   Log-Likelihood:                -74.005
No. Observations:                  32   AIC:                             154.0
Df Residuals:                      29   BIC:                             158.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.6863      1.715     23.141      0.0

The above script solves for the regression via OLS. Note that these are the binary solutions.

In [44]:
matrix2 = np.vstack((intercept, np.array(weight), np.array(cyl))).T
model = sm.OLS(mpg, matrix2).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                     70.91
Date:                Tue, 11 Oct 2022   Prob (F-statistic):           6.81e-12
Time:                        20:47:21   Log-Likelihood:                -74.005
No. Observations:                  32   AIC:                             154.0
Df Residuals:                      29   BIC:                             158.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.6863      1.715     23.141      0.0

The above script works for solving the linear regression for the actual values of weight and number of cylinders. It does not take interaction into account.



In [69]:
from google.colab import files

data = pd.read_excel("8 A1C Data.xlsx")

data.head()

alc = np.array(data['A1c'].values.tolist())
net = data['Netflix'].values.tolist()
dogs = data['Dog Owner'].values.tolist()
both = []
netbi = []
dogsbi = []

for k in range(len(net)):
  if net[k] == 'Yes':
    netbi.append(1)
  else:
    netbi.append(0)
  if dogs[k] == 'Yes':
    dogsbi.append(1)
  else:
    dogsbi.append(0)

both = np.array(netbi) * np.array(dogsbi)

intercept2 = np.ones(len(net))

matrix3 = np.vstack((intercept2, netbi, dogsbi, both)).T

tmp = inv(np.matmul(np.transpose(matrix3), matrix3))

INTERCEPT, SLOPEnet, SLOPEdogs, SLOPEboth = tmp @ np.transpose(matrix3) @ alc

print('Intercept is:', INTERCEPT, "slope netflix is:", SLOPEnet, 'slope dogs is:', SLOPEdogs, 'slope interaction is:', SLOPEboth)

model3 = sm.OLS(alc, matrix3).fit()

print(model3.summary())

infl = model3.get_influence()

print(infl.summary_table())

Intercept is: 5.6055868539310465 slope netflix is: 2.6466384272577983 slope dogs is: -1.8790852366239417 slope interaction is: -2.4860883017033863
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.978
Method:                 Least Squares   F-statistic:                     467.6
Date:                Tue, 11 Oct 2022   Prob (F-statistic):           5.19e-24
Time:                        21:10:21   Log-Likelihood:                -1.9604
No. Observations:                  32   AIC:                             11.92
Df Residuals:                      28   BIC:                             17.78
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.

Thanks for a great lecture!!